# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [1]:
# Instalamos el modelo predictivo Prophet
!python -m pip install prophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [2]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [16]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-06-28'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2730,2023-06-23,"$29,896.38","$31,389.54","$29,845.21","$30,695.47","$30,695.47",24115570085
2731,2023-06-24,"$30,708.74","$30,804.15","$30,290.15","$30,548.70","$30,548.70",12147822496
2732,2023-06-25,"$30,545.15","$31,041.27","$30,327.94","$30,480.26","$30,480.26",12703464114
2733,2023-06-26,"$30,480.52","$30,636.03","$29,955.74","$30,271.13","$30,271.13",16493186997
2734,2023-06-27,"$30,274.32","$31,006.79","$30,236.65","$30,688.16","$30,688.16",16428827944


In [18]:
# Validamos que no hay datos vacíos
btc_df.isnull().sum()



Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [24]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
btc_df[['Date', 'Open']]
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds",
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns = new_names)

In [25]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [26]:
df.tail()

,ds,y
2730,2023-06-23,"$29,896.38"
2731,2023-06-24,"$30,708.74"
2732,2023-06-25,"$30,545.15"
2733,2023-06-26,"$30,480.52"
2734,2023-06-27,"$30,274.32"


In [27]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = ""
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [28]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet()


In [29]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzgukz2co/pbgbw7xw.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpzgukz2co/3pxnmukb.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=55855', 'data', 'file=/tmp/tmpzgukz2co/pbgbw7xw.json', 'init=/tmp/tmpzgukz2co/3pxnmukb.json', 'output', 'file=/tmp/tmpzgukz2co/prophet_model1_ez126e/prophet_model-20230628100725.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:07:25 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:07:26 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [30]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(365)
future.tail()

,ds
3095,2024-06-22
3096,2024-06-23
3097,2024-06-24
3098,2024-06-25
3099,2024-06-26


In [31]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05
...,...
3095,2024-06-22
3096,2024-06-23
3097,2024-06-24
3098,2024-06-25


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [32]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [33]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
3095,2024-06-22,"$-5,236.05","$-18,701.12","$7,661.35"
3096,2024-06-23,"$-5,425.75","$-18,455.67","$7,783.58"
3097,2024-06-24,"$-5,641.73","$-19,227.14","$7,971.11"
3098,2024-06-25,"$-5,848.20","$-18,554.51","$7,021.47"
3099,2024-06-26,"$-6,116.59","$-18,743.97","$7,144.75"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [34]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)